# Market order execution

For this example, let's consider in the BTC-USD pair, executing a series BUY market order against the SELL side of the resting orderbook.

- $(p_i, A_i)$: a SELL limit order, where $p_i$ is the limit price (as the units of USD that's equal in value as 1 unit of BTC); $A_i$ is the order's size (as units of BTC).
- $\{ (p_i, A_i) \mid i = 1, 2, \dots, N \}$: the SELL side of the resting orderbook, sorted by price-time priority. Specifically, $p_1 \leq p_2 \leq p_3 \leq \dots \leq p_N$.
- $(B_j, s_j)$: a BUY market order, where $B_j$ is the budget (as units of USD); $s_j$ is the maximum slippage (defined below).
- $\{ (B_j, s_j) \mid j = 1, 2, \dots, M \}$: the series of BUY market orders, sorted by priority.

Maximum slippage works as follows: suppose the best ask price at the time of executing market order $(B_j, s_j)$ is $p_{\mathrm{best}}$, then this market order's _average execution price_ must not be worse than $p_{\mathrm{max}} = (1 + s_j) p_{\mathrm{best}}$. Note it's possible that the market order is executed against a limit order whose limit price $p_i > p_{\mathrm{max}}$, as long as the _average_ price over all limit order consumed is no greater than $p_{\mathrm{max}}$.

Note that $A_i$ and $B_j$ are integers, denoted in the minimum indivisible units of BTC and USD, respectively. (Here in the Python prototype, however, for simplicity, we use floating point numbers.)

The algorithm is a function that has the following inputs and outputs:

- Inputs:
  - $\{ (p_i, A_i) \mid i = 1, 2, \dots, N \}$
  - $\{ (B_j, s_j) \mid j = 1, 2, \dots, M \}$
- Outputs:
  - $\mathtt{limitOrderBtcSold}_{\{1, 2, \dots, N\}}$ (for each limit order, the amount of BTC sold)
  - $\mathtt{limitOrderUsdBought}_{\{1, 2, \dots, N\}}$ (for each limit order, the amount of USD bought)
  - $\mathtt{marketOrderUsdSold}_{\{1, 2, \dots, M\}}$ (for each market order, the amount of USD sold)
  - $\mathtt{marketOrderBtcBought}_{\{1, 2, \dots, M\}}$ (for each market order, the amount of BTC bought)

The algorithm is as follows:

- $\mathtt{limitOrderBtcSold}_{\{1, 2, \dots, N\}} \gets [0, 0, \dots, 0]$
- $\mathtt{limitOrderUsdBought}_{\{1, 2, \dots, N\}} \gets [0, 0, \dots, 0]$
- $\mathtt{marketOrderUsdSold}_{\{1, 2, \dots, M\}} \gets [0, 0, \dots, 0]$
- $\mathtt{marketOrderBtcBought}_{\{1, 2, \dots, M\}} \gets [0, 0, \dots, 0]$
- $i \gets 1$ (index tracking the current limit order)
- $j \gets 1$ (index tracking the current market order)
- $A_{\mathrm{remain}} \gets A_i$ (track the remaining BTC amount in the current limit order)
- $B_{\mathrm{remain}} \gets B_j$ (track the remaining USD amount in the current market order)
- $p_{\mathrm{max}} \gets (1 + s_j) p_i$ (the maximum average execution price for the current market order)
- Loop:
  - First, compute how many BTC is available to buy. This is determined by the remaining amount in either the limit or the market order, whichever is smaller:
  - $A_{\mathrm{available}} \gets \mathrm{min} \left( A_{\mathrm{remain}}, \left\lfloor \frac{B_{\mathrm{remain}}}{p_i} \right\rfloor \right)$
  - If $p_i \leq p_{\mathrm{max}}$:
    - In this case, we buy as much BTC as possible.
    - $A_{\mathrm{buy}} \gets A_{\mathrm{available}}$
  - Else (that is, $p_i > p_{\mathrm{max}}$)
    - In this case, we buy up to the point where the average execution price reaches $p_{\mathrm{max}}$.
    - $A_{\mathrm{buy}} \gets \mathrm{min} \left( A_{\mathrm{available}}, \left\lfloor \frac{p_{\mathrm{max}} \mathtt{marketOrderBtcBought}_j - \mathtt{marketOrderUsdSold}_j}{p_i - p_{\mathrm{max}}} \right\rfloor \right)$
  - $\mathtt{limitOrderBtcSold}_i \mathrel{{+}{=}} A_{\mathrm{buy}}$
  - $\mathtt{limitOrderUsdBought}_i \mathrel{{+}{=}} p_i A_{\mathrm{buy}}$
  - $\mathtt{marketOrderUsdSold}_j \mathrel{{+}{=}} p_i A_{\mathrm{buy}}$
  - $\mathtt{marketOrderBtcBought}_j \mathrel{{+}{=}} A_{\mathrm{buy}}$
  - $A_{\mathrm{remain}} \mathrel{{-}{=}} A_{\mathrm{buy}}$
  - $B_{\mathrm{remain}} \mathrel{{-}{=}} p_i A_{\mathrm{buy}}$
  - If either a) $B_{\mathrm{remain}} = 0$, or b) $B_{\mathrm{remain}} \neq 0$ and $A_{\mathrm{remain}} \neq 0$
    - (a) means this market order is completely consumed; (b) means we can't execute this market any more without exceeding $p_{\mathrm{max}}$ (the price limit). Either way, move on to the next market order.
    - If $j = M$:
      - **Break**
    - $j \gets j + 1$
    - $B_{\mathrm{remain}} \gets B_j$
    - If $A_{\mathrm{remain}} \neq 0$:
      - $p_{\mathrm{max}} \gets (1 + s_j) p_i$
    - Else if $i < N$:
      - $p_{\mathrm{max}} \gets (1 + s_j) p_{i+1}$
    - Else:
      - **Break**
  - If $A_{\mathrm{remain}} = 0$:
    - This limit order is completely consumed. Move on to the next limit order.
    - if $i = N$:
      - **Break**
    - $i \gets i + 1$
    - $A_{\mathrm{remain}} \gets A_i$
- Return $\mathtt{limitOrderBtcSold}_{\{1, 2, \dots, N\}}$, $\mathtt{limitOrderUsdBought}_{\{1, 2, \dots, N\}}$, $\mathtt{marketOrderUsdSold}_{\{1, 2, \dots, M\}}$, $\mathtt{marketOrderBtcBought}_{\{1, 2, \dots, M\}}$

In [1]:
import pprint

In [2]:
def execute_market_orders(limit_orders, market_orders):
    outcome = {
        'limit': [{'btc_sold': 0, 'usd_bought': 0} for i in range(len(limit_orders))],
        'market': [{'usd_sold': 0, 'btc_bought': 0} for j in range(len(market_orders))],
    }

    i = 0
    j = 0

    A_remain = limit_orders[i]['btc_amount']
    B_remain = market_orders[j]['usd_amount']

    p_i = limit_orders[i]['price']
    p_max = (1 + market_orders[j]['maximum_slippage']) * p_i

    while True:
        A_buy = min(A_remain, B_remain / limit_orders[i]['price'])
        
        if p_i > p_max:
            A_buy = min(A_buy, (p_max * outcome['market'][j]['btc_bought'] - outcome['market'][j]['usd_sold']) / (p_i - p_max))

        B_sell = A_buy * limit_orders[i]['price']

        outcome['limit'][i]['btc_sold'] += A_buy
        outcome['limit'][i]['usd_bought'] += B_sell
        outcome['market'][j]['usd_sold'] += B_sell
        outcome['market'][j]['btc_bought'] += A_buy

        A_remain -= A_buy
        B_remain -= B_sell

        print(f'i={i}, j={j}, p_max={p_max}, p_i={p_i}, A_buy={A_buy}, B_sell={B_sell}, A_remain={A_remain}, B_remain={B_remain}')

        if B_remain == 0 or A_remain != 0:
            j += 1
            if j == len(market_orders):
                print('----------------------- reached the end of market orders -----------------------')
                break

            if A_remain != 0:
                p_max = (1 + market_orders[j]['maximum_slippage']) * p_i
            elif i + 1 < len(limit_orders):
                p_max = (1 + market_orders[j]['maximum_slippage']) * limit_orders[i + 1]['price']
            else:
                print('----------------------- reached the end of limit orders ------------------------')
                break

            B_remain = market_orders[j]['usd_amount']

        if A_remain == 0:
            i += 1
            if i == len(limit_orders):
                print('----------------------- reached the end of limit orders ------------------------')
                break

            p_i = limit_orders[i]['price']
            A_remain = limit_orders[i]['btc_amount']

    for i in range(len(limit_orders)):
        outcome['limit'][i]['remaining'] = limit_orders[i]['btc_amount'] - outcome['limit'][i]['btc_sold']
        if outcome['limit'][i]['btc_sold'] != 0:
            outcome['limit'][i]['clearing_price'] = outcome['limit'][i]['usd_bought'] / outcome['limit'][i]['btc_sold']

    for j in range(len(market_orders)):
        outcome['market'][j]['remaining'] = market_orders[j]['usd_amount'] - outcome['market'][j]['usd_sold']
        if outcome['market'][j]['btc_bought'] != 0:
            outcome['market'][j]['clearing_price'] = outcome['market'][j]['usd_sold'] / outcome['market'][j]['btc_bought']

    return outcome

In [3]:
# 1 limit order, 1 market order, exactly equal amounts.
pprint.pp(execute_market_orders(
    limit_orders=[
        {
            'btc_amount': 100,
            'price': 200,
        },
    ],
    market_orders=[
        {
            'usd_amount': 20_000,
            'maximum_slippage': 0.005,
        },
    ],
))

i=0, j=0, p_max=200.99999999999997, p_i=200, A_buy=100, B_sell=20000, A_remain=0, B_remain=0
----------------------- reached the end of market orders -----------------------
{'limit': [{'btc_sold': 100,
            'usd_bought': 20000,
            'remaining': 0,
            'clearing_price': 200.0}],
 'market': [{'usd_sold': 20000,
             'btc_bought': 100,
             'remaining': 0,
             'clearing_price': 200.0}]}


In [4]:
# 2 limit orders, 1 market order.
# The market order fully consumes the 1st limit order, but partially consumes the 2nd limit order.
pprint.pp(execute_market_orders(
    limit_orders=[
        {
            'btc_amount': 50,
            'price': 200,
        },
        {
            'btc_amount': 50,
            'price': 205,
        }
    ],
    market_orders=[
        {
            'usd_amount': 20_000,
            'maximum_slippage': 0.005,
        },
    ],
))

i=0, j=0, p_max=200.99999999999997, p_i=200, A_buy=50, B_sell=10000, A_remain=0, B_remain=10000
i=1, j=0, p_max=200.99999999999997, p_i=205, A_buy=12.499999999999456, B_sell=2562.4999999998886, A_remain=37.50000000000054, B_remain=7437.500000000111
----------------------- reached the end of market orders -----------------------
{'limit': [{'btc_sold': 50,
            'usd_bought': 10000,
            'remaining': 0,
            'clearing_price': 200.0},
           {'btc_sold': 12.499999999999456,
            'usd_bought': 2562.4999999998886,
            'remaining': 37.50000000000054,
            'clearing_price': 205.0}],
 'market': [{'usd_sold': 12562.499999999889,
             'btc_bought': 62.49999999999946,
             'remaining': 7437.500000000111,
             'clearing_price': 200.99999999999997}]}


In [5]:
# 2 limit orders, 1 market order.
# Market order 1 fully consumes limit order 1, but partially consumes the limit order 2.
# Market order 2 fully consumes limit order 2, with some USD remaining.
pprint.pp(execute_market_orders(
    limit_orders=[
        {
            'btc_amount': 50,
            'price': 200,
        },
        {
            'btc_amount': 50,
            'price': 205,
        }
    ],
    market_orders=[
        {
            'usd_amount': 20_000,
            'maximum_slippage': 0.005,
        },
        {
            'usd_amount': 20_000,
            'maximum_slippage': 0.005,
        }
    ],
))

i=0, j=0, p_max=200.99999999999997, p_i=200, A_buy=50, B_sell=10000, A_remain=0, B_remain=10000
i=1, j=0, p_max=200.99999999999997, p_i=205, A_buy=12.499999999999456, B_sell=2562.4999999998886, A_remain=37.50000000000054, B_remain=7437.500000000111
i=1, j=1, p_max=206.02499999999998, p_i=205, A_buy=37.50000000000054, B_sell=7687.500000000111, A_remain=0.0, B_remain=12312.499999999889
----------------------- reached the end of limit orders ------------------------
{'limit': [{'btc_sold': 50,
            'usd_bought': 10000,
            'remaining': 0,
            'clearing_price': 200.0},
           {'btc_sold': 50.0,
            'usd_bought': 10250.0,
            'remaining': 0.0,
            'clearing_price': 205.0}],
 'market': [{'usd_sold': 12562.499999999889,
             'btc_bought': 62.49999999999946,
             'remaining': 7437.500000000111,
             'clearing_price': 200.99999999999997},
            {'usd_sold': 7687.500000000111,
             'btc_bought': 37.500000000